## Introduction

This notebook will introduce some properties of **cubic Bézier curves** as implemented in the [**Cairo**](http://cairographics.org/) 2D graphics library. Access to Cairo in Python will be done via the **Qahirah** ([GitLab](https://gitlab.com/ldo/qahirah), [BitBucket](https://bitbucket.org/ldo17/qahirah)) high-level binding.

The stages of the exposition are as follows:
* [Constructing The Bézier Curve](#Constructing-The-B%C3%A9zier-Curve)
* [More Complex Curves](#More-Complex-Curves)
* [Rendering A Cubic Bézier](#Rendering-A-Cubic-B%C3%A9zier)
* [Joining Several Bézier Curves](#Joining-Several-B%C3%A9zier-Curves)

The first cell sets up some common definitions which will be reused later. Note the following useful routines:

* `reset()` — reinitializes the drawing context
* `display()` — displays the current drawing

In [ ]:
from ipywidgets.widgets import interact
from IPython.display import display_png
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Path, \
    Rect, \
    Vector

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (400, 400)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
       .save()
       .set_source_colour(Colour.grey(.95))
       .paint()
       .restore()
       .select_font_face(family = "serif", slant = CAIRO.FONT_SLANT_NORMAL, weight = CAIRO.FONT_WEIGHT_NORMAL)
       .set_font_size(12)
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

reset()

The next cell defines routines for displaying the example Bézier curve with its control points clearly marked.

In [ ]:
def draw_marker(pos : Vector, label : str, colour : Colour) :
    "draws a marker into ctx at pos and gives it a label."
    pos = Vector.from_tuple(pos)
    marker_size = 8
    marker_radius = 6
    marker_gap = 2
    (ctx
         .save()
         .set_line_width(1)
         .new_path()
         .arc
           (
              centre = pos,
              radius = marker_radius,
              angle1 = 0,
              angle2 = qah.circle,
              negative = False
           )
         .set_source_colour(colour)
         .fill_preserve()
         .set_source_colour(Colour.grey(0))
         .stroke()
    )
    for i in range(4) :
        angle = i / 4 * qah.circle
        (ctx
             .move_to(pos + Vector(marker_gap, 0).rotate(angle))
             .line_to(pos + Vector(marker_size, 0).rotate(angle))
        )
    #end for
    ctx.stroke()
    if label != None :
        (ctx
             .move_to(pos + Vector(5, 5))
             .set_source_colour(Colour.grey(0))
             .show_text(label)
             .restore()
        )
    #end if
#end draw_marker

def draw_curve(p0, p1, p2, p3) :
    (ctx
        .save()
        .set_line_width(hull_line_width)
        .new_path()
        .move_to(p0)
        .line_to(p1)
        .line_to(p2)
        .line_to(p3)
        .set_source_colour(hull_lines_colour)
        .stroke()
        .move_to(p0)
        .curve_to(p1, p2, p3)
        .set_source_colour(curve_colour)
        .set_line_width(curve_line_width)
        .stroke()
        .restore()
    )
#end draw_curve

def draw_curve_labelled(p0, p1, p2, p3) :
    draw_curve(p0, p1, p2, p3)
    draw_marker(p0, "p0", marker_colour_1)
    draw_marker(p1, "p1", marker_colour_1)
    draw_marker(p2, "p2", marker_colour_1)
    draw_marker(p3, "p3", marker_colour_1)
#end draw_curve_labelled

p0, p1, p2, p3 = Vector(50, 270), Vector(200, 300), Vector(300, 150), Vector(280, 50)
hull_lines_colour = Colour.grey(0)
curve_colour = Colour.from_hsva((0.7, 1, 1))
marker_colour_1 = Colour.grey(0.5)
marker_colour_2 = Colour.grey(0.75)
marker_colour_0 = Colour.grey(1, 0.5)
hull_line_width = 1
curve_line_width = 3
t_range = (0.0, 1.0, 0.01)
reset()
draw_curve_labelled(p0, p1, p2, p3)
display()


## Constructing The Bézier Curve

But how is a Bézier curve built up? It is best understood in terms of a *parametric* representation: the curve is traced out by varying a real parameter $t$ which goes over the range $[0, 1]$. As it does so, it computes the current point on the curve in terms of the fixed *control points*.

To illustrate this, let us start with a simple straight line, which we may consider to be a *Bézier curve of order 1*. Execute the cell, then use the slider to vary the value of $t$, and see how the current curve point (darker marker) varies smoothly between the control points (lighter markers), which mark the endpoints of the line.

In [ ]:
@interact(t = t_range)
def display_b1(t) :
    "interactive Bézier curve of order 1 (straight line)."
    reset()
    p0 = Vector(100, 320)
    p1 = Vector(330, 100)
    (ctx
         .save()
         .set_line_width(curve_line_width)
         .new_path()
         .move_to(p0)
         .line_to(p1)
         .set_source_colour(curve_colour)
         .set_line_width(curve_line_width)
         .stroke()
         .restore()
    )
    draw_marker(p0, "p0", marker_colour_1)
    draw_marker(p1, "p1", marker_colour_1)
    draw_marker((1 - t) * p0 + t * p1, "p(t)", marker_colour_0)
    display()
#end display_b1

To put it mathematically, if the control points are $\overrightarrow{p_0}$ and $\overrightarrow{p_1}$, then the curve point $\overrightarrow{p(t)}$ is defined in terms of the parameter $t$ as

$$\overrightarrow{p(t)} = (1 - t)\overrightarrow{p_0} + t\overrightarrow{p_1}$$

Thus, when $t = 0$, the curve point coincides exactly with the control point $\overrightarrow{p_0}$, and when $t = 1$, it coincides exactly with the control point $\overrightarrow{p_1}$.

Next, let us try a Bëzier curve of order 2, or a *quadratic* Bëzier. This is defined in terms of three control points $\overrightarrow{p_0}$, $\overrightarrow{p_1}$ and $\overrightarrow{p_2}$, and is built up from the order-1 Bézier in an interesting way.

In [ ]:
@interact(t = t_range)
def display_b2(t) :
    "interactive Bézier curve of order 2."
    reset()
    p0 = Vector(80, 300)
    p1 = Vector(280, 305)
    p2 = Vector(310, 80)
    q0 = (1 - t) * p0 + t * p1
    q1 = (1 - t) * p1 + t * p2
    (ctx
        .save()
        .set_line_width(hull_line_width)
        .set_source_colour(hull_lines_colour)
        .new_path()
        .move_to(p0)
        .line_to(p1)
        .line_to(p2)
        .stroke()
        .move_to(q0)
        .line_to(q1)
        .stroke()
        .move_to(p0)
        .curve_to(*Path.cubify(p0, p1, p2)[1:])
          # Cairo doesn’t directly support constructing quadratic Béziers.
          # But Qahirah provides a convenience function to express a quadratic as a cubic.
        .set_source_colour(curve_colour)
        .set_line_width(curve_line_width)
        .stroke()
        .restore()
    )
    draw_marker(p0, "p0", marker_colour_1)
    draw_marker(p1, "p1", marker_colour_1)
    draw_marker(p2, "p2", marker_colour_1)
    draw_marker(q0, "q0(t)", marker_colour_2)
    draw_marker(q1, "q1(t)", marker_colour_2)
    draw_marker((1 - t) * q0 + t * q1, "p(t)", marker_colour_0)
    display()
#end display_b2

What is happening here? First of all, there are two separate order-1 Bézier lines being constructed:

$$\overrightarrow{q_0(t)} = (1 - t)\overrightarrow{p_0} + t\overrightarrow{p_1}$$
$$\overrightarrow{q_1(t)} = (1 - t)\overrightarrow{p_1} + t\overrightarrow{p_2}$$

Then these two curve points are *themselves* being used to construct another order-1 Bézier:

$$\overrightarrow{p(t)} = (1 - t)\overrightarrow{q_0(t)} + t\overrightarrow{q_1(t)}$$

Only because the control points of this one are not fixed, but are themselves moving (being functions of $t$), the net result is that the final curve point traces out, not a straight line, but a quadratic curve.

***

This composition can be applied again, to construct a *cubic* (order-3) Bézier curve with control points $\overrightarrow{p_0}$, $\overrightarrow{p_1}$, $\overrightarrow{p_2}$ and $\overrightarrow{p_3}$. By taking successive pairs of control points, we make 3 order-1 Bézier lines:

$$\overrightarrow{q_0(t)} = (1 - t)\overrightarrow{p_0} + t\overrightarrow{p_1}$$
$$\overrightarrow{q_1(t)} = (1 - t)\overrightarrow{p_1} + t\overrightarrow{p_2}$$
$$\overrightarrow{q_2(t)} = (1 - t)\overrightarrow{p_2} + t\overrightarrow{p_3}$$

on top of which we make 2 order-2 Béziers:

$$\overrightarrow{r_0(t)} = (1 - t)\overrightarrow{q_0(t)} + t\overrightarrow{q_1(t)}$$
$$\overrightarrow{r_1(t)} = (1 - t)\overrightarrow{q_t(t)} + t\overrightarrow{q_2(t)}$$

which in turn are used to build our cubic Bézier:

$$\overrightarrow{p(t)} = (1 - t)\overrightarrow{r_0(t)} + t\overrightarrow{r_1(t)}$$

In [ ]:
@interact(t = t_range)
def display_b3(t) :
    "interactive Bézier curve of order 3."
    reset()
    draw_curve_labelled(p0, p1, p2, p3)
    q0 = (1 - t) * p0 + t * p1
    q1 = (1 - t) * p1 + t * p2
    q2 = (1 - t) * p2 + t * p3
    r0 = (1 - t) * q0 + t * q1
    r1 = (1 - t) * q1 + t * q2
    (ctx
        .save()
        .set_source_colour(Colour.grey(0.5))
        .set_line_width(1)
        .move_to(q0)
        .line_to(q1)
        .line_to(q2)
        .move_to(r0)
        .line_to(r1)
        .stroke()
        .restore()
    )
    draw_marker(q0, "q0(t)", marker_colour_2)
    draw_marker(q1, "q1(t)", marker_colour_2)
    draw_marker(q2, "q2(t)", marker_colour_2)
    draw_marker(r0, "r0(t)", marker_colour_2)
    draw_marker(r1, "r1(t)", marker_colour_2)
    draw_marker((1 - t) * r0 + t * r1, "p(t)", marker_colour_0)
    display()
#end display_b3

## More Complex Curves

It should be easy enough to see how to extend the above construction to higher-order Béziers. However, Cairo doesn’t support them, and they are rarely seen in graphics software. The main reason is that, in a Bézier curve, **every control point affects the entire curve**. Thus, in a higher-order Bézier, trying to juggle lots of control points makes it quite hard to shape the curve the exact way you want.

Like most graphics software, Cairo builds more complex curves out of multiple cubic Bézier and straight-line segments. Each segment responds to its own control points, and the interactions between segments can be kept easier to manage.

By the way, if it wasn’t obvious from the above, **higher-order Béziers are strict supersets of lower-order ones**. That is, a cubic Bézier can exactly express any curve that a quadratic can (and then some), and a quadratic Bézier (and hence a cubic Bézier) can also exactly represent any straight line. Cairo provides straight-line segments as a separate path element type more for convenience and efficiency than anything else.

## Rendering A Cubic Bézier

The usual technique (as used by Cairo) to draw a cubic Bézier curve is to approximate it by a series of straight lines. This is where the **tolerance** setting comes into play in a Cairo context: it specifies the maximum deviation from the true curve that is allowed for the straight-line approximations.

The actual construction of the straight-line sequence is done using **de Casteljau’s algorithm**. This decomposes the original Bézier into a series of pieces that, when drawn in sequence, are geometrically equivalent to the original curve. Even though the orders of the sub-curves remain the same (splitting a cubic Bézier yields cubic Béziers), the smaller they are, the closer they become to straight lines joining the endpoints. if a sub-curve lies everywhere within the tolerance of such a straight line, it is replaced by that line. Otherwise, the algorithm is applied recursively to split the sub-curve into two sub-sub-curves. And so on.

In [ ]:
def split_curve(p0 : Vector, p1 : Vector, p2 : Vector, p3 : Vector) :
    "splits a cubic Bézier curve defined by the given control points into" \
    " two sub-curves according to de Casteljau’s construction. Returns a tuple" \
    " of two 4-tuples of coefficients."
    pa0 = p0
    pb3 = p3
    pa1 = (p0 + p1) / 2
    pb2 = (p2 + p3) / 2
    p12 = (p1 + p2) / 2
    pa2 = (pa1 + p12) / 2
    pb1 = (p12 + pb2) / 2
    pa3 = (pa2 + pb1) / 2
    pb0 = pa3
    return \
        (
            (pa0, pa1, pa2, pa3),
            (pb0, pb1, pb2, pb3),
        )
#end split_curve

def curve_variance(p0 : Vector, p1 : Vector, p2 : Vector, p3 : Vector) :
    "given the four control points of a cubic Bézier curve, computes the square of" \
    " its deviation from a straight line between the endpoints.\n" \
    "Algorithm adapted from Cairo source code, src/cairo-spline.c."

    def closest_dist(pn) :
        delta_pn = pn - p0
        delta_line = p3 - p0
        v = delta_line.dot(delta_line)
        u = delta_pn.dot(delta_line)
        if u < 0 :
            pass
        elif u >= v :
            delta_pn -= delta_line
        else :
            delta_pn -= u / v * delta_line
        #end if
        return \
            delta_pn
    #end closest_dist

#begin curve_variance
    if p0 != p3 :
        delta_p1 = closest_dist(p1)
        delta_p2 = closest_dist(p2)
    else :
        delta_p1 = p1 - p0
        delta_p2 = p2 - p0
    #end if
    return \
        max(delta_p1.dot(delta_p1), delta_p2.dot(delta_p2))
#end curve_variance

def draw_curve_unlabelled(p0, p1, p2, p3) :
    draw_curve(p0, p1, p2, p3)
    draw_marker(p0, None, marker_colour_1)
    draw_marker(p1, None, marker_colour_1)
    draw_marker(p2, None, marker_colour_1)
    draw_marker(p3, None, marker_colour_1)
#end draw_curve_unlabelled

def draw_decomposed_curve(p0, p1, p2, p3, tolerance) :
    "draws an approximation of the Bézier curve with the specified control points," \
    " decomposed into straight-line segments within the specified tolerance."

    markers = [p0]
    
    def draw_decomposed_subcurve(p0, p1, p2, p3) :
        if curve_variance(p0, p1, p2, p3) > tolerance * tolerance :
            subcurve1, subcurve2 = split_curve(p0, p1, p2, p3)
            draw_decomposed_subcurve(*subcurve1)
            draw_decomposed_subcurve(*subcurve2)
        else :
            # close enough to a straight line
            ctx.line_to(p3)
            markers.append(p3)
        #end if
    #end draw_decomposed_subcurve

#begin draw_decomposed_curve
    (ctx.new_path()
        .move_to(p0)
        .set_source_colour(curve_colour)
        .set_line_width(curve_line_width)
    )
    draw_decomposed_subcurve(p0, p1, p2, p3)
    ctx.stroke()
    for pos in markers :
        draw_marker(pos, None, marker_colour_1)
    #end for
#end draw_decomposed_curve

reset()
ctx.translate((-30, -30))
draw_curve_unlabelled(p0, p1, p2, p3)
ctx.translate((50, 50))
for subcurve in split_curve(p0, p1, p2, p3) :
    draw_curve_unlabelled(*subcurve)
#end for
ctx.translate((50, 50))
draw_decomposed_curve(p0, p1, p2, p3, 1)
display()

The following interactive example lets you try out the effect of different tolerance values on the decomposition:

In [ ]:
@interact(tolerance = (1.0, 10.0, 2.0))
def display_decomposed(tolerance) :
    reset()
    ctx.translate((50, 50))
    draw_decomposed_curve(p0, p1, p2, p3, tolerance)
    display()
#end display_decomposed

## Joining Several Bézier Curves

As previously mentioned, Cairo, like most 2D graphics software, lets you build up more complex curves by joining multiple cubic Bézier segments (and straight lines) together.

Since a Bézier curve always passes through its end control points, it is easy enough to ensure that adjacent segments *join* by letting them share adjacent endpoints (even if the join is not smooth). Mathematically, this is called $C^0$ or $G^0$ (zero-order) [*continuity*](https://en.wikipedia.org/wiki/Smoothness).

In [ ]:
pa0, pa1, pa2, pa3 = Vector(20, 330), Vector(130, 350), Vector(180, 300), Vector(190, 230)
pb0, pb1, pb2, pb3 = pa3, Vector(260, 190), Vector(310, 130), Vector(340, 180)

reset()
draw_curve_unlabelled(pa0, pa1, pa2, pa3)
draw_curve_unlabelled(pb0, pb1, pb2, pb3)
display()

To ensure a smooth join (that means the curve does not suddenly change direction at the join), not only must the adjacent endpoints be in common, but the neighbouring control points on each side *must lie on the same straight line*—this is called first-order ($C^1$ or $G^1$) *continuity*. If they are also the same distance from the endpoint, then the continuity goes to $C^2$/$G^2$ (second order). This means that the amount of curvature does not suddenly change across the join.

This second-order continuity has a number of practical applications. For example, if the curve were a railway track, and you were a passenger on a train carriage crossing that point at a constant speed, you would not experience any sudden lurching change in acceleration. For another example, if two separate curve segments in the shape of a car body meet with less than second-order continuity, then reflections will appear broken across the join; they will only be smooth if the continuity is second-order.

In [ ]:
pa0, pa1, pa2, pa3 = Vector(20, 330), Vector(130, 350), Vector(180, 300), Vector(190, 230)
pb0, pb1, pb2, pb3 = pa3, 2 * pa3 - pa2, Vector(310, 130), Vector(340, 180)

reset()
draw_curve_unlabelled(pa0, pa1, pa2, pa3)
draw_curve_unlabelled(pb0, pb1, pb2, pb3)
display()